# Data Preparation

## Select Data - Eliminar Columnas
De la seccion de entendimiento de los datos, vimos que hay columnas que realmente no aportan mucha información, importancia o significancia al alcance de este proyecto, por lo que en esta parte, se eliminaran para contar unicamente con los datos que se trabajaran, estas columnas son:

- municipio_hechos : Al tener el 99% de sus datos como NaN y no aportar informacion
- competencia : Al tener un unico valor
- coordenadas (latitud y longitud) : El proyecto se enfocara en las alcaldias de la Ciudad de Mexico y el valor de coordenadas no estan dentro de
este scope
- colonia_datos : No esta dentro del scope de este proyecto
- fgj_colonia_registro : No esta dentro del scope de este proyecto

In [ ]:
import pandas as pd

data_set = pd.read_csv("data/sourceData.csv")

In [ ]:
columns_to_remove = ['municipio_hechos', 'competencia', 'latitud', 'longitud', 'colonia_datos', 'fgj_colonia_registro']
data_set = data_set.drop(columns_to_remove, axis=1)

data_set.head()

## Outliers
Las columnas que cuentan con valores outliers son:
- edad: Valores mayores a 120 (258, 361 y 369) y 1899 valores de 0
- año_hechos: Un valor de 222 y valores del siglo pasado

In [ ]:
valores_edad = data_set.groupby('Edad')['idCarpeta'].count()
valores_ahecho = data_set.groupby('Anio_hecho')['idCarpeta'].count()

### Outliers - Edad
El approach a seguir es el obtener un promedio de edad entre un limite inferior (1) y un limite superior (120)
Despues de obtener este promedio, reemplazamos los outliers (valores de 0, 258, 361 y 369) con este promedio, ya que el numero de valores en 0 es significativo (1899)

In [ ]:
limite_inferior = 1
limite_superior = 120
edad_en_rango = data_set.loc[data_set['Edad'].between(limite_inferior, limite_superior), 'Edad']
promedio_edad = edad_en_rango.mean()
print(promedio_edad)

promedio_edad = round(promedio_edad)
print(promedio_edad)

In [ ]:
data_set['Edad'] = data_set['Edad'].mask(~data_set['Edad'].between(limite_inferior, limite_superior), promedio_edad)
print(data_set.groupby('Edad')['idCarpeta'].count())

## Outliers - Año Hechos
Se borran las filas con valores menores a 2000

In [ ]:
data_set = data_set[data_set['Anio_hecho'] >= 2000]
print(data_set.groupby('Anio_hecho')['idCarpeta'].count())

## Cleaning Data
Se borran las filas con valores a CDMX y FUERA DE CDMX para las alcaldias
Ya que este proyecto se enfoca unicamente en alcaldias dentro de la ciudad

In [ ]:
data_set = data_set[~data_set['alcaldia_hechos'].isin(['CDMX', 'FUERA DE CDMX'])]
print(data_set['alcaldia_hechos'].unique())

## Missing Data
Las columnas con valores NaN son las siguientes:
- Sexo : 180,609 registros
- TipoPersona : 6512 registros
- CalidadJuridica: 1 registro
- Anio_hecho, Mes_hecho, FechaHecho : 370 registros
- HoraHecho : 361 registros
- HoraInicio : 1
- colonia_datos : 69,602 registros

In [ ]:
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

### Grupo 1 - Columnas con menos de 1000 registros Nan
El approach para este grupo es borrar los registros, ya que son de un numero muy bajo
Las columnas a borrar sus registros con valores nulos son:
- CalidadJuridica
- Anio_hecho
- Mes_hecho
- FechaHecho
- HoraHecho
- HoraInicio

In [ ]:
data_set = data_set.dropna(subset=['CalidadJuridica', 'Anio_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho', 'HoraInicio'])
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

### Grupo 2 - Columnas con mas de 1000 registros Nan
Para estas columnas, el approach es sustituir los valores Nulos con la palabra 'Desconocido'
Las columnas a ser modificadas son:
- Sexo
- TipoPersona
- colonia_datos

In [ ]:
data_set[['Sexo', 'TipoPersona']] = data_set[['Sexo', 'TipoPersona']].fillna('Desconocido')
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

## Data Transformation
En esta seccion, se modificaran las columnas de formato HH:MM:SS a HH:MM
Estas columnas son:
- HoraHecho
- HoraInicio

In [ ]:
data_set[['HoraHecho', 'HoraInicio']] = data_set[['HoraHecho', 'HoraInicio']].apply(pd.to_datetime)
data_set['HoraHecho'] = data_set['HoraHecho'].dt.strftime('%H:%M')
data_set['HoraInicio'] = data_set['HoraInicio'].dt.strftime('%H:%M')
data_set.head()

### Crear Columna Dias Transcurridos
Con base a las columnas FechaInicio y FechaHecho, obtenemos los dias transcurridos entre la fecha en que se reporto el delito y la fecha en que ocurrio, para darnos una idea de si estos dias tienen un patron respecto a sexo o edad (hipotesis).

Una vez realizado esto, las columnas de Anio_inicio, Mes_inicio y FechaInicio no seran de utilidad ya que para realizar las predicciones las fechas importantes son las de los hechos

In [ ]:
fechas_inicio = pd.to_datetime(data_set['FechaInicio'], format="%d/%m/%Y")
fechas_hechos = pd.to_datetime(data_set['FechaHecho'], format="%d/%m/%Y")

data_set.insert(13, 'Dias_transcurridos', (fechas_inicio - fechas_hechos).dt.days)

# Borrar fechas inicio
columns_to_remove = ['Anio_inicio', 'Mes_inicio', 'FechaInicio']
data_set = data_set.drop(columns_to_remove, axis=1)

data_set.head()

### Crear Columna Semanas
Con base a la columna FechaHecho, tomaremos el valor mas viejo (el minimo de la fecha) y comenzaremos un counter de semana en uno, por cada semana transcurrida aumentaremos el counter a un valor de 1 para crear un timelapse semanal desde el hecho mas viejo al mas reciente para posteriormente realizar predicciones de delitos por semana.
1. Obtener el valor minimo de FechaHecho
2. Comenzar el counter de semanas
1. Por cada semana transcurrida aumentar el counter
2. Asignar el valor de counter correspondiente a la FechaHecho en una columna 'abs_semana'
3. Eliminar la columna Anio_hecho ya que no aporta mucho

Se descubrio que la fecha mas vieja del data_set es el primero de enero del 2000, si se quiere realizar algun modelo predictivo desde una fecha mas reciente a esta, por ejemplo, desde el 2015, se debera crear un offset para evitar errores, un ejemplo seria:
- La semana uno original es 01/01/2000
- El modelo predictivo que se desea crear (ejemplo) es a partir del 01/01/2015
- Han pasada 782 semanas entre ambas fechas por lo que la fecha 01/01/2015 tiene un valor de 782 en la columna count_semanas
- Se crea un offset de 782 es decir (count_semanas) - 781 para tomar a la fecha 01/01/2015 como la mas vieja
- NOTA: Ajustar el numero de 782 por la fecha en que se desea comenzar el modelo

In [ ]:
oldest_date = fechas_hechos.min()
data_set.insert(11, 'count_semanas', (fechas_hechos - oldest_date).dt.days // 7 + 1)
data_set = data_set.drop(['Anio_hecho'], axis=1)
data_set.head()

### Crear Columna Rango Edad
Con base al valor de la columna Edad, se creara una variable categorica descriptiva generando 6 rangos diferentes de edad (Niño, Adolescente, y más), con la finalidad de tener un entendimiento mejor de los datos y poder interpretar mejor los resultados del modelo a crear posteriormente

In [ ]:
from mapeo_poblacion import obtener_rango_edad

data_set.insert(loc=5, column='rango_edad', value=data_set['Edad'].apply(obtener_rango_edad))
data_set.head()

### Columnas Sexo y tipo de persona Numerico

In [ ]:
from mapeo_poblacion import mapeo_sexo, mapeo_tipo_persona

data_set.insert(loc=5, column='Sexo_num', value=data_set['Sexo'].apply(mapeo_sexo))
data_set.insert(loc=8, column='TipoPersona_num', value=data_set['TipoPersona'].apply(mapeo_tipo_persona))

## Data Integration

### Poblacion Alcaldias
En esta fase nos centraremos en agregar información externa proveniente de otras bases de datos y fuentes de información
Para esto, integraremos datos obtenidos de las estadisticas de Poblacion del INEGI para mostrar la poblacion por alcaldia, agregando una nueva columna de poblacion dependiendo el valor de la columna de alcaldia_hechos
Para realizar este mapeo entre la alcaldia de los hechos y su poblacion, se llevo a cabo un script en Python

In [ ]:
from mapeo_poblacion import get_population

data_set.insert(loc=14, column='poblacion_alcaldia', value=data_set['alcaldia_hechos'].apply(get_population))
data_set.head()

### Categorización de Delitos
Actualmente el dataset cuenta con 16 tipos diferentes de Categorias y 310 tipos diferentes de Delitos, al ser tantos valores unicos se dificulta el
analisis conciso de este tipo de delitos y los 16 tipos de categoria son algo ambigüos como "Delito de bajo Impacto".

Para esto se ha utilizado de un algoritmo open source que emplea técnicas avanzadas de procesamiento de lenguaje natural, para clasificar distintos
tipos de delitos en funcion de su nivel de riesgo y severidad. Este algoritmo analiza y evalua la gravedad de cada delito, permitiendo establecer
rangos que van desde situaciones de menor riesgo (valor de 1) hasta aquellas que presentan un mayor nivel de peligro (valor de 10). El objetivo de esta
herramienta es proporcionar una clasificacion precisa y confiable de los delitos.

De esta forma la columna 'Categoria' deja de ser util y puede ser eliminada.

In [ ]:
numero_categorias = len(pd.unique(data_set['Categoria']))
numero_delitos = len(pd.unique(data_set['Delito']))
print(f"Categorias Distintas =  {numero_categorias}; Delitos Distintos =  {numero_delitos}")

In [ ]:
from mapeo_crimen import get_crime_score

data_set.insert(loc=2, column='riesgo_delito', value=data_set['Delito'].apply(get_crime_score))
data_set = data_set.drop('Categoria', axis=1)
data_set.head()

In [ ]:
data_set.to_csv('data/data_preparation.csv', index=False)

In [40]:
dias_mayores = data_set[data_set['Dias_transcurridos'] > 30]
counts = dias_mayores['Delito'].value_counts()
sorted_counts = counts.sort_values(ascending=False)
sorted_counts.to_csv('data/dias_mayores.csv')

In [42]:
average_values = data_set.groupby('riesgo_delito')['poblacion_alcaldia'].mean()

# Print the average values
print(average_values)

riesgo_delito
1     762041.500638
2     731761.466405
3     771705.766788
4     778342.345900
5     704733.602606
6     813364.053267
7     791849.750052
8     827455.489872
9     901290.752079
10    877608.999224
Name: poblacion_alcaldia, dtype: float64
